In [ ]:
!ls ../working/

Подключим все необходимые библиотеки для обработки данных

In [ ]:
import gc # сборщик мусора для удаления ненужных данных в оперативной памяти
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings("ignore") # отключаем предупреждения
pd.set_option('display.float_format', lambda x: '%.5f' % x) # переключим режим отображения чисел

# Проверка данных

Загружаем наши данные, просмотрим их и проверяем на наличие пустых значений

In [ ]:
# path = r"drive/My Drive/Colab Notebooks/data_sales_prediction/"
path = r"../input/competitive-data-science-predict-future-sales/"

items_cat = pd.read_csv(path + "item_categories.csv")
items = pd.read_csv(path + "items.csv")
train = pd.read_csv(path + "sales_train.csv")
sample_sub = pd.read_csv(path + "sample_submission.csv")
shops = pd.read_csv(path + "shops.csv")
test = pd.read_csv(path + "test.csv")

In [ ]:
items_cat.head()

In [ ]:
items.head()

In [ ]:
train.head()

In [ ]:
sample_sub.head()

In [ ]:
shops.head()

In [ ]:
test.head()

In [ ]:
items_cat.isnull().sum()

In [ ]:
items.isnull().sum()

In [ ]:
train.isnull().sum()

In [ ]:
shops.isnull().sum()

In [ ]:
test.isnull().sum()

# Feature engineering

Приведем наши данные к необходимому формату, объединим таблицы. Удалим из train выборки то чего нет в test выборке. Добавим новые признаки. Приведем целевую переменную item_cnt_day к формату 0, 20.

In [ ]:
test_shops = test.shop_id.unique() # в train выборке у нас есть магазины и товары которых нет в test выборке
train = train[train.shop_id.isin(test_shops)] # поэтому мы их удалим
test_items = test.item_id.unique()
train = train[train.item_id.isin(test_items)]

In [ ]:
def split_city(str):
  return str.split(sep=" ", maxsplit=1)[0]

def split_shop(str):
  return str.split(sep=" ", maxsplit=1)[1]

def split_item_cat1(str):
  return str.split(sep="-", maxsplit=1)[0]

def split_item_cat2(str):
  splitted = str.split(sep="-", maxsplit=1)
  if len(splitted) == 1:
    return "No info"
  else:
    return splitted[1]

def prepare_data(data): # функция для объединения таблиц и создания новых признаков из старых
  full_items = items.merge(items_cat, left_on="item_category_id", right_on="item_category_id")
  full_data = data.merge(shops, left_on="shop_id", right_on="shop_id").merge(full_items, left_on="item_id", right_on="item_id")
  del full_items
  full_data['city'] = full_data['shop_name'].apply(split_city)
  full_data['new_shop_name'] = full_data['shop_name'].apply(split_shop)
  full_data['item_cat1'] = full_data['item_category_name'].apply(split_item_cat1)
  full_data['item_cat2'] = full_data['item_category_name'].apply(split_item_cat2)
  full_data.drop(['shop_id', 'item_id', 'shop_name', 'item_name', 'item_category_id', 'item_category_name'], axis=1, inplace=True)
  return full_data

In [ ]:
%%time
new_train = prepare_data(train.copy())
new_test = prepare_data(test.copy())

In [ ]:
new_test['date_block_num'] = 34 # добавляем порядковый номер месяца в test
new_test.drop(['ID'], axis=1, inplace=True)
new_train.drop(['date'], axis=1, inplace=True)
new_train['item_cnt_day'] = new_train['item_cnt_day'].clip(0, 20) # преобразуем значения item_cnt_day в необходимый формат
new_train['month'] = new_train['date_block_num'] % 12 # добавляем номер месяца в train
new_test['month'] = new_test['date_block_num'] % 12 # добавляем номер месяца в test
new_train.drop(['item_price'], axis=1, inplace=True)

In [ ]:
gc.collect()

In [ ]:
new_train

In [ ]:
new_test

# Получение информации о данных

In [ ]:
new_train.info()

In [ ]:
new_test.info()

In [ ]:
new_train.describe()

# Визуализация

Посмотрим количество заказов по городам.

In [ ]:
plt.figure(figsize=(20, 5))
city = sns.countplot(x='city', data=new_train)
city.set_xticklabels(city.get_xticklabels(), rotation=45);

Распределение покупок по магазинам.

In [ ]:
plt.figure(figsize=(30, 5))
shop_viz = sns.countplot(x='new_shop_name', data=new_train)
shop_viz.set_xticklabels(shop_viz.get_xticklabels(), rotation=45);

Распределение покупок по глобальным категориям.

In [ ]:
plt.figure(figsize=(20, 5))
item_name = sns.countplot(x='item_cat1', data=new_train)
item_name.set_xticklabels(item_name.get_xticklabels(), rotation=45);

Распределение покупок по локальным категориям.


In [ ]:
plt.figure(figsize=(35, 5))
item_label = sns.countplot(x='item_cat2', data=new_train)
item_label.set_xticklabels(item_label.get_xticklabels(), rotation=45);

Распределение покупок по месяцам.

In [ ]:
plt.figure(figsize=(20, 5))
sns.countplot(x='month', data=new_train);

# Подготавливаем данные для обучения моделей

In [ ]:
X_train = new_train.drop(['item_cnt_day'], axis=1) # разделение на X и Y
Y_train = new_train['item_cnt_day']
X_test = new_test

In [ ]:
cat_features = ['city', 'new_shop_name', 'item_cat1', 'item_cat2']

def into_numbers(data): # приводим к необходимому формату категориальные признаки
  num_data = pd.concat([data, pd.get_dummies(data['city'])], axis=1)
  num_data = pd.concat([num_data, pd.get_dummies(data['item_cat1'])], axis=1)
  num_data = pd.concat([num_data, pd.get_dummies(data['item_cat2'])], axis=1)
  num_data = pd.concat([num_data, pd.get_dummies(data['new_shop_name'])], axis=1)
  num_data.drop(cat_features, axis=1, inplace=True)
  return num_data

In [ ]:
%%time
X_train_num = into_numbers(X_train.copy())
X_test_num = into_numbers(X_test.copy())

В train выборке отсутствуют 3 категории товаров, которые есть в test выборке, поэтому мы добавим их вручную в train выборке.

In [ ]:
X_train_num[' Гарнитуры/Наушники'] = 0
X_train_num['PC ' ] = 0
X_train_num['Игры MAC '] = 0

Для XGBoost столбцы признаков должны быть в одном порядке и у train, и test выборок, поэтому мы отсортируем столбцы, чтобы получить одинаковый порядок столбцов.

In [ ]:
X_train_num = X_train_num.reindex(sorted(X_train_num.columns), axis=1)
X_test_num = X_test_num.reindex(sorted(X_test_num.columns), axis=1)

Из-за того, что LightGBM не принимает не ascii-символы придется делать транслитерацию названий колонок.

In [ ]:
def transliterate(name):
   """
   Автор: LarsKort
   Дата: 16/07/2011; 1:05 GMT-4;
   """
   # Словарь с заменами
   slovar = {'а':'a','б':'b','в':'v','г':'g','д':'d','е':'e','ё':'e',
      'ж':'zh','з':'z','и':'i','й':'i','к':'k','л':'l','м':'m','н':'n',
      'о':'o','п':'p','р':'r','с':'s','т':'t','у':'u','ф':'f','х':'h',
      'ц':'c','ч':'cz','ш':'sh','щ':'scz','ъ':'','ы':'y','ь':'','э':'e',
      'ю':'u','я':'ja', 'А':'A','Б':'B','В':'V','Г':'G','Д':'D','Е':'E','Ё':'E',
      'Ж':'ZH','З':'Z','И':'I','Й':'I','К':'K','Л':'L','М':'M','Н':'N',
      'О':'O','П':'P','Р':'R','С':'S','Т':'T','У':'U','Ф':'F','Х':'H',
      'Ц':'C','Ч':'CZ','Ш':'SH','Щ':'SCH','Ъ':'','Ы':'y','Ь':'','Э':'E',
      'Ю':'U','Я':'YA',',':'','?':'',' ':'_','~':'','!':'','@':'','#':'',
      '$':'','%':'','^':'','&':'','*':'','(':'',')':'','-':'','=':'','+':'',
      ':':'',';':'','<':'','>':'','\'':'','"':'','\\':'','/':'','№':'',
      '[':'',']':'','{':'','}':'','ґ':'','ї':'', 'є':'','Ґ':'g','Ї':'i',
      'Є':'e', '—':''}
        
   # Циклически заменяем все буквы в строке
   for key in slovar:
      name = name.replace(key, slovar[key])
   return name

In [ ]:
eng_cols = {}
for i in X_train_num.columns:
    eng_cols[str(i)] = transliterate(i)

In [ ]:
X_train_num.rename(columns=eng_cols, inplace=True)
X_test_num.rename(columns=eng_cols, inplace=True)

In [ ]:
gc.collect()

# Создание моделей

In [ ]:
import xgboost
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Разбиваем данные train выборки на выборку для обучения и отложенную выборку для проверки качества моделей.

In [ ]:
X_train_check, X_test_check, y_train_check, y_test_check = train_test_split(X_train_num, Y_train, test_size=0.33, random_state=42) # нормализованные данные с использованием метода обработки категориальных признаков - get_dummies

Удалим старые данные, чтобы освободить ОЗУ.

In [ ]:
del X_train_num
del Y_train
del X_train
del X_test
del items_cat
del items
del train
del sample_sub
del shops
del test
del new_train
del new_test
gc.collect()

In [ ]:
model_error = {}

Пробуем разные модели "из коробки".

# LinearRegression

In [ ]:
%%time
model_lr = LinearRegression()
model_lr.fit(X_train_check, y_train_check)

In [ ]:
pred_lr = model_lr.predict(X_test_check)
lr_rmse = np.sqrt(mean_squared_error(y_test_check, pred_lr))
model_error['LinearRegression'] = lr_rmse
print(lr_rmse)

# SGDRegressor

In [ ]:
%%time
model_sgd = SGDRegressor()
model_sgd.fit(X_train_check, y_train_check)

In [ ]:
pred_sgd = model_sgd.predict(X_test_check)
sgd_rmse = np.sqrt(mean_squared_error(y_test_check, pred_sgd))
model_error['SGDRegressor'] = sgd_rmse
print(sgd_rmse)

# XGBoost

In [ ]:
%%time
model_xgboost = xgboost.XGBRegressor()
model_xgboost.fit(X_train_check, y_train_check)

In [ ]:
pred_xgb = model_xgboost.predict(X_test_check)
xgboost_rmse = np.sqrt(mean_squared_error(y_test_check, pred_xgb))
model_error['XGBoost'] = xgboost_rmse
print(xgboost_rmse)

In [ ]:
xgboost.plot_importance(model_xgboost) # график важности признаков

# RandomForest

In [ ]:
%%time
model_rf = RandomForestRegressor()
model_rf.fit(X_train_check, y_train_check)

In [ ]:
pred_rf = model_rf.predict(X_test_check)
rf_rmse = np.sqrt(mean_squared_error(y_test_check, pred_rf))
model_error['RandomForest'] = rf_rmse
print(rf_rmse)

# LightGBM

In [ ]:
model_lgbm = LGBMRegressor()
model_lgbm.fit(X_train_check, y_train_check)

In [ ]:
pred_lgbm = model_lgbm.predict(X_test_check)
lgbm_rmse = np.sqrt(mean_squared_error(y_test_check, pred_lgbm))
model_error['LightGBM'] = lgbm_rmse
print(lgbm_rmse)

Посмотрим на то как алгоритмы работают "из коробки"

In [ ]:
full_rmse = 0
for key, value in model_error.items():
  full_rmse += value
  print("RMSE ошибка модели {} - {}".format(key, str(value)))
print("Среднее качество моделей - {}".format(str(full_rmse / len(model_error))))

# GridSearch

Очевидно мы можем улучшить качество моделей с помощью перебора гиперпараметров по сетке.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs_model_error = {}

In [ ]:
%%time
params_sgd = {
    'epsilon':np.linspace(0.2, 0.01, 6), # константа случайного шума модели
    'alpha': [0, 0.1, 1, 5], # коэффициент регуляризации
    'eta0': np.linspace(0.2, 0.01, 6), # скорость обучения
    'penalty':['l1', 'l2', 'elasticnet'], # вид регуляризации
    'shuffle':[False]
}

best_sgd = GridSearchCV(SGDRegressor(), params_sgd, cv=None, refit=True, n_jobs=-1)
best_sgd.fit(X_train_check, y_train_check)

best_sgd_params = best_sgd.best_params_
print(best_sgb_params)

pred_best_sgd = best_sgd.predict(X_test_check)
best_rmse_sgd = np.sqrt(mean_squared_error(y_test_check, pred_best_sgd))
gs_model_error['SGDRegressor'] = best_rmse_sgd
print(best_rmse_sgd)

In [ ]:
%%time
params_lgbm = {
    'num_leaves': [30, 50, 100, 1000], # количество листьев
    'max_depth':[-1, 6, 10], # максимальная глубина деревьев
    'learning_rate':np.linspace(0.3, 0.01, 10), # скорость обучения
    'lambda_l1':[0, 0.1, 1, 5], # коэффициент l1 регуляризации
    'lambda_l2':[0, 0.1, 1, 5], # коэффициент l2 регуляризации
    'num_iterations':[100, 200, 500], # количество итераций
    'device_type':['gpu'] # на чем обучается модель
}

best_lgbm = GridSearchCV(LGBMRegressor(), params_lgbm, cv=None, refit=True)
best_lgbm.fit(X_train_check, y_train_check)

best_lgbm_params = best_lgbm.best_params_
print(best_lgbm_params)

pred_best_lgbm = best_lgbm.predict(X_test_check)
best_rmse_lgbm = np.sqrt(mean_squared_error(y_test_check, pred_best_lgbm))
gs_model_error['LightGBM'] = best_rmse_lgbm
print(best_rmse_lgbm)

In [ ]:
%%time
params_xgboost = {
    'booster'['dart', 'gbtree'], # вид алгоритмов бустинга
    'eta': np.linspace(0.5, 0.01, 6), # скорость обучения
    'max_depth':[6, 8, 10], # максимальная глубина деревьев
    'lambda': [0, 0.1, 1, 5], # коэффициент l2 регуляризации
    'alpha':[0, 0.1, 1, 5], # коэффициент l1 регуляризации
    'tree_method':['gpu_hist'], # вид конструкции деревьев
    'gpu_id':[0] # номер видеокарты
    'eval_metric':['rmse'] # используемая метрика ошибки
}

best_xgboost = GridSearchCV(xgboost.XGBRegressor(), params_xgboost, cv=None, refit=True)
best_xgboost.fit(X_train_check, y_train_check)

best_xgboost_params = best_xgboost.best_params_
print(best_xgboost_params)

pred_best_xgboost = best_xgboost.predict(X_test_check)
best_rmse_xgboost = np.sqrt(mean_squared_error(y_test_check, pred_best_xgboost))
gs_model_error['XGBoost'] = best_rmse_xgboost
print(best_rmse_xgboost)

In [ ]:
%%time
params_rf = {
    'n_estimators': [10, 30, 50], # количество деревьев в ансабле
    'max_depth':[None, 6, 8, 10], # глубина деревьев
    'max_features': ['auto', 'sqrt'], # количество признаков используемых при обучении одного дерева
    'n_jobs':[-1] # количество потоков
}

best_rf = GridSearchCV(RandomForestRegressor(), params_rf, cv=None, refit=True)
best_rf.fit(X_train_check, y_train_check)

best_rf_params = best_rf.best_params_
print(best_rf_params)

pred_best_rf = best_rf.predict(X_test_check)
best_rmse_rf = np.sqrt(mean_squared_error(y_test_check, pred_best_rf))
gs_model_error['RandomForest'] = best_rmse_rf
print(best_rmse_rf)

Посмотрим на результаты нашего перебора.

In [ ]:
full_rmse_gs = 0
for key, value in gs_model_error.items():
  full_rmse_gs += value
  print("RMSE ошибка модели {} - {}".format(key, str(value)))
print("Среднее качество моделей - {}".format(str(full_rmse_gs / len(gs_model_error))))

Мы добились результата лучше, чем с параметрами "из коробки". 

In [ ]:
gc.collect()

# Stacking models

С помощью техники стекинга моделей попробуем сделать нашу ошибку еще меньше.

In [ ]:
from sklearn.ensemble import StackingRegressor

In [ ]:
%%time
estimators = [
              ('lr', LinearRegression(n_jobs=-1)),
              ('sgd', SGDRegressor(**best_sgd_params)),
              ('xgboost', xgboost.XGBRegressor(**best_xgboost_params)),
              ('rf', RandomForestRegressor(**best_rf_params))
              ]

stack = StackingRegressor(estimators=estimators, final_estimator=LGBMRegressor(**best_lgbm_params))
stack.fit(X_train_check, y_train_check)

In [ ]:
pred_stack = stack.predict(X_test_check)
stack_error = np.sqrt(mean_squared_error(y_test_check, pred_stack))
print("RMSE ошибка при стекинге моделей - {}".format(stack_error))

In [ ]:
gc.collect()

# Create submission

In [ ]:
pred_for_sub = stack.predict(X_test_num)
len(pred_for_sub)

In [ ]:
sub = pd.DataFrame({'ID':test.ID, 'item_cnt_month':pred_for_sub})
sub

In [ ]:
sub.to_csv("sub.csv", index=False)